In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv
load_dotenv()

from rs import Message, Config, chat
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))

def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"
    
    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_36.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


You arrived at an old self-storage facility by invitation from the manager, who has an storage room up for auction due to the death of its owner. It is not your first rodeo with these forms of auctions, like a box of chocolate, no one knows of the potential treasure that hides behind the cold roller shutter. As you and other interested buyers gathered around the auctioneer, a worker carrying a bolt cutter snapped off the rusty lock, unveiling a storage room cluttered with various old furniture and antiquities. Without any opportunity for the bidders to take a clear look, the auctioneer began the bid at $50. 

What is your bid?

possible actions:
1. $100.
2. $150.
3. $200.


I select option 2. TURN: 1 


You raise your hand confidently and shout, "$150!" The auctioneer nods in your direction, acknowledging your bid. The crowd murmurs, and another bidder, a well-dressed man with a keen eye for value, counters with "$200!" The tension among the bidders is palpable, as everyone tries to gauge the potential value hidden within the cluttered storage room. 

possible actions:
1. Increase your bid to $250.
2. Increase your bid to $300.
3. Hold your bid and wait to see if the price goes higher.

I select option 3. TURN: 2 


You decide to hold your bid, observing the situation closely. The auctioneer's voice fills the air, "We have $200! Do I hear $250?" A moment of silence follows, as the bidders seem hesitant to push the price further. The well-dressed man who previously outbid you appears confident, almost smug, as he waits for the auctioneer to call the sale in his favor. 

Suddenly, a new voice from the back calls out, "$250!" The crowd parts slightly, revealing a middle-aged woman with a determined look in her eyes. The auctioneer nods, "We have $250! Do I hear $300?"

possible actions:
1. Increase your bid to $300.
2. Increase your bid to $350.
3. Hold your bid and observe the situation further.

I select option 2. TURN: 3 


With a deep breath, you call out, "$350!" Your voice cuts through the murmurs of the crowd, drawing all eyes to you. The auctioneer acknowledges your bid with a nod, "We have $350! Do I hear $400?" The atmosphere is charged with anticipation as the other bidders weigh their options.

The well-dressed man hesitates, exchanging a quick, uncertain glance with the middle-aged woman who previously bid. It seems your bold move has made them reconsider their strategy. The auctioneer's voice grows more urgent, "Going once at $350... Going twice..."

possible actions:
1. Prepare to celebrate your potential win.
2. Increase your bid to secure the win.
3. Wait for the auctioneer's final call.

I select option 3. TURN: 4 


You hold your breath, waiting for the auctioneer's final call. The tension is almost tangible, as the other bidders remain silent, their earlier confidence replaced by hesitation. "Going twice at $350... And sold to the gentleman for $350!" The auctioneer's hammer comes down, and a mixture of relief and excitement washes over you. You've just secured the storage room, potentially filled with hidden treasures, for $350.

As the crowd begins to disperse, the auctioneer approaches you to finalize the paperwork and collect payment. You hand over the $350, feeling a mix of anticipation and curiosity about what lies within the cluttered confines of your newly acquired storage room.

round summary: Kyle Jackson, the player, successfully won the bid for the storage room at $350 after a tense auction. He decided to hold his bid at strategic moments, ultimately outbidding the competition without exceeding his budget.